# Grossmünster #311/918, scene 0

In [ ]:
!s5cmd --no-sign-request ls s3://megascenes/reconstruct/311/918/colmap/0/

In [ ]:
!s5cmd --no-sign-request ls -H s3://megascenes/databases/databases/ #main/311/918/

In [ ]:
!s5cmd --no-sign-request cp s3://megascenes/reconstruct/311/918/colmap/0/* ./reconstruct/311/918/colmap/0/
!s5cmd --no-sign-request cp s3://megascenes/reconstruct_aux/311/918/colmap/0/* ./reconstruct_aux/311/918/colmap/0/
!s5cmd --no-sign-request cp s3://megascenes/databases/main/311/918/* ./databases/main/311/918/

!ls -lah ./reconstruct/311/918/colmap/0/

In [ ]:
import pycolmap
import pandas as pd
import sqlite3
import os


reconstruction = pycolmap.Reconstruction("./reconstruct/311/918/colmap/0/")
reconstruction.summary()

In [ ]:
reconstruction

In [ ]:
for image_id, image in reconstruction.images.items():
    print(image_id, image)
    break

In [ ]:

#reconstruction.images[3].projection_center()
#reconstruction.images[3].summary()
#reconstruction.images[3].viewing_directio()


print(reconstruction.images[3].cam_from_world)
print(reconstruction.images[3].projection_center())
print(reconstruction.images[3].summary())
print(reconstruction.images[3].viewing_direction())

In [ ]:
img = reconstruction.images[3].todict()
del img["points2D"]
coords = img["cam_from_world"]["translation"]
img

In [ ]:
from colmap.scripts.python.read_write_model import read_images_binary

all_bin_images = read_images_binary("reconstruct/311/918/colmap/0/images.bin")
len(all_bin_images)

In [ ]:
img = all_bin_images[3]

['camera_id',
 'count',
 'id',
 'index',
 'name',
 'point3D_ids',
 'qvec',
 'qvec2rotmat',
 'tvec',
 'xys']
print(img.camera_id)
print(img.qvec)
print(img.qvec2rotmat())
print(img.tvec)
print(img.xys.shape)

https://colmap.github.io/format.html#images-txt   
The coordinates of the projection/camera center are given by -R^t * T, where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector.

In [ ]:
import numpy as np
from colmap.scripts.python.read_write_model import Image
import pandas as pd
import plotly.express as px

def get_coordinates_from_image(img:Image) -> np.array:
    # https://colmap.github.io/format.html#images-txt
    Rt = img.qvec2rotmat().transpose() # https://automaticaddison.com/how-to-convert-a-quaternion-to-a-rotation-matrix/
    T = img.tvec

    coordinates = np.matmul(-Rt, T)
    return coordinates

get_coordinates_from_image(img)

In [ ]:
image_coordinates = []
image_index = []
for index, image in all_bin_images.items():
    image_coordinates.append(get_coordinates_from_image(image))
    image_index.append(index)

image_coordinates = np.vstack(image_coordinates)
print(image_coordinates.shape)

x = -image_coordinates[:,2]
y = image_coordinates[:,0]
z = -image_coordinates[:,1]
xyz = np.concatenate([[x],[y],[z]], axis=0).T
print(xyz.shape)
plot_df = xyz_array_to_plot_df(xyz) # z, x, y
plot_xyz(plot_df)

In [ ]:
def xyz_array_to_plot_df(arr):
    df = pd.DataFrame(arr, columns=["x", "y", "z"])

    df["color"] = "black"

    origin = pd.DataFrame({
        "x": [0],
        "y": [0], 
        "z": [0],
        "color": "red"
    }, index=[99999])

    df = pd.concat([df, origin])

    return df

def plot_xyz(df):
    fig = px.scatter_3d(
            plot_df, 
            x="x", 
            y="y", 
            z="z", 
            color ="color", 
            )

    fig.update_traces(marker=dict(size=1.5))
    fig.show()

plot_df = xyz_array_to_plot_df(image_coordinates)
plot_xyz(plot_df)

# SQLite has only raw data not usable

In [ ]:

con = sqlite3.connect(os.path.normpath('./databases/main/311/918/database.db'))

with con:    
    cur = con.cursor()    
    cur.execute('select image_id, prior_tx, prior_ty, prior_tz from images;')
    data = cur.fetchall()   
con.close()

data[:5]

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns=["idx", "tx", "ty", "tz"])
df.set_index("idx", inplace=True)
df = df[df["tx"].notnull()]

df.loc[mask_low, "tz"] = 450
df.loc[mask_high, "tz"] = df.loc[mask_high, "tz"] /1000
df[df["ty"] < 8.5] = df.describe().loc["50%"]


df["color"] = "black"

origin = pd.DataFrame({
    "tx": [0],
    "ty": [0], 
    "tz": [0],
    "color": "red"
}, index=[99999])

#df = pd.concat([df, origin])
df = pd.concat([df])

df

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
            df, 
            x="tx", 
            y="ty", 
            z="tz", 
            color ="color", 
            )


fig.show()

# Alles mumpitz below

In [ ]:
all_cam_coords = []
cam_ids = []
for id, camera in reconstruction.images.items():
    cam_ids.append(id)
    all_cam_coords.append(camera.todict()["cam_from_world"]["translation"])

cam_position = np.vstack(all_cam_coords)
cam_position.shape

In [ ]:
df = pd.DataFrame(cam_position, columns=["x", "y", "z"], index=cam_ids)

df["color"] = "black"

origin = pd.DataFrame({
    "x": [0],
    "y": [0], 
    "z": [0],
    "color": "red"
}, index=[99999])

df = pd.concat([df, origin])
df

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')

x, y, z = np.split(cam_position, 3, 1)

ax.set_box_aspect([1.0, 1.0, 1.0])
ax.scatter(x, y, z)


ax.scatter(0, 0, 0, "red")
plt.show()

In [ ]:

df = pd.DataFrame(cam_position, columns=["x", "y", "z"])

df["color"] = "black"

origin = pd.DataFrame({
    "x": [0],
    "y": [0], 
    "z": [0],
    "color": "red"
})

df = pd.concat([df, origin])
df

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
            df, 
            x="x", 
            y="y", 
            z="z", 
            color ="color", 
            )


fig.show()